In [10]:
import torch
import numpy as np
import pickle


# Extract Euler angles from rotation matrix
def get_euler_angles_from_rotation_matrix(rot_matrix):
    rot_matrix = rot_matrix.permute(0, 2, 1)
    yaw = torch.arcsin(-rot_matrix[:, 2, 0])
    near_pi_over_2 = torch.isclose(torch.cos(yaw), torch.tensor(0.0), atol=1e-6)

    pitch = torch.where(
        ~near_pi_over_2,
        torch.atan2(rot_matrix[:, 2, 1], rot_matrix[:, 2, 2]),
        torch.atan2(rot_matrix[:, 1, 2], rot_matrix[:, 1, 1])
    )

    roll = torch.where(
        ~near_pi_over_2,
        torch.atan2(rot_matrix[:, 1, 0], rot_matrix[:, 0, 0]),
        torch.zeros_like(yaw)
    )

    pitch = pitch * 180 / torch.pi
    yaw = yaw * 180 / torch.pi
    roll = roll * 180 / torch.pi

    return pitch, yaw, roll


def get_rotation_matrix(pitch_, yaw_, roll_):
    """ the input is in degree
    """
    # transform to radian
    pitch = pitch_ / 180 * torch.pi
    yaw = yaw_ / 180 * torch.pi
    roll = roll_ / 180 * torch.pi

    device = pitch.device

    if pitch.ndim == 1:
        pitch = pitch.unsqueeze(1)
    if yaw.ndim == 1:
        yaw = yaw.unsqueeze(1)
    if roll.ndim == 1:
        roll = roll.unsqueeze(1)

    # calculate the euler matrix
    bs = pitch.shape[0]
    ones = torch.ones([bs, 1]).to(device)
    zeros = torch.zeros([bs, 1]).to(device)
    x, y, z = pitch, yaw, roll

    rot_x = torch.cat([
        ones, zeros, zeros,
        zeros, torch.cos(x), -torch.sin(x),
        zeros, torch.sin(x), torch.cos(x)
    ], dim=1).reshape([bs, 3, 3])

    rot_y = torch.cat([
        torch.cos(y), zeros, torch.sin(y),
        zeros, ones, zeros,
        -torch.sin(y), zeros, torch.cos(y)
    ], dim=1).reshape([bs, 3, 3])

    rot_z = torch.cat([
        torch.cos(z), -torch.sin(z), zeros,
        torch.sin(z), torch.cos(z), zeros,
        zeros, zeros, ones
    ], dim=1).reshape([bs, 3, 3])

    rot = rot_z @ rot_y @ rot_x
    return rot.permute(0, 2, 1)  # transpose

# Convert yaw, pitch, and roll to a rotation matrix
def euler_angles_to_rotation_matrix(pitch, yaw, roll):
    PI = np.pi
    # Convert to torch tensors and add batch dimension
    pitch_ = torch.tensor([pitch], dtype=torch.float32)
    yaw_ = torch.tensor([yaw], dtype=torch.float32)
    roll_ = torch.tensor([roll], dtype=torch.float32)

    # Get rotation matrix using provided function
    R = get_rotation_matrix(pitch_, yaw_, roll_)

    # Convert to numpy and reshape to (1,3,3)
    R = R.cpu().numpy().astype(np.float32)

    return R


# Function to extract the full 208-dimensional vector from frame data
def extract_full_vector(frame_data):
    c_d_eyes = frame_data['c_d_eyes_lst'][0].reshape(-1)  # 2 values
    c_d_lip = frame_data['c_d_lip_lst'][0].reshape(-1)    # 1 value

    driving_template = frame_data['driving_template_dct']
    c_eyes = frame_data['c_d_eyes_lst'][0].reshape(-1) # 2 values
    c_lip = frame_data['c_d_lip_lst'][0].reshape(-1)    # 1 value

    motion = driving_template['motion'][0]
    scale = np.array(motion['scale']).reshape(-1)         # 1 value
    t = motion['t'].reshape(-1)                           # 3 values
    R = motion['R'].reshape(1, 3, 3)                      # 9 values in matrix form
    exp = motion['exp'].reshape(-1)                       # 63 values
    x_s = motion['x_s'].reshape(-1)                       # 63 values
    kp = motion['kp'].reshape(-1)                         # 63 values actual value now becomes 202

    # Convert R to pitch, yaw, and roll using the function
    pitch, yaw, roll = get_euler_angles_from_rotation_matrix(torch.tensor(R))
    euler_angles = np.array([pitch.item(), yaw.item(), roll.item()])

    if not np.array_equal(c_d_eyes, c_eyes):
        print("Eyes arrays not equal")
    if not np.array_equal(c_d_lip, c_lip):
        print("Lip arrays not equal")

    # print(c_d_eyes.shape, c_d_lip.shape, c_eyes.shape, c_lip.shape, scale.shape, t.shape, euler_angles.shape, exp.shape, x_s.shape, kp.shape)
    # print("(2,) (1,) (2,) (1,) (1,) (3,) (3,) (63,) (63,) (63,)")
    # 202 values

    # Combine the components into a full vector excluding R
    vector = np.concatenate([c_d_eyes, c_d_lip, c_eyes, c_lip, scale, t, euler_angles, exp, x_s, kp])

    return vector

def unflatten_vector(avg_vector):
    # Convert flattened vector back to original format
    c_d_eyes = np.array(avg_vector[0:2], dtype=np.float32).reshape(1, 2)
    c_d_lip = np.array(avg_vector[2:3], dtype=np.float32).reshape(1, 1)
    c_eyes = np.array(avg_vector[3:5], dtype=np.float32).reshape(1, 2)
    c_lip = np.array(avg_vector[5:6], dtype=np.float32).reshape(1, 1)
    scale = np.array(avg_vector[6:7], dtype=np.float32).reshape(1, 1)
    t = np.array(avg_vector[7:10], dtype=np.float32).reshape(1, 3)

    # Convert to rotation matrix and update
    R = euler_angles_to_rotation_matrix(avg_vector[10], avg_vector[11], avg_vector[12])

    # Expression, shape and keypoint parameters
    exp = np.array(avg_vector[13:76], dtype=np.float32).reshape(1, 21, 3)
    x_s = np.array(avg_vector[76:139], dtype=np.float32).reshape(1, 21, 3)
    kp = np.array(avg_vector[139:202], dtype=np.float32).reshape(1, 21, 3)

    # Return dictionary in original format
    return {
        'c_d_eyes_lst': c_d_eyes,
        'c_d_lip_lst': c_d_lip,
        'driving_template_dct': {
            'motion': [{
                'scale': scale,
                'R': R,
                't': t,
                'c_eyes_lst': c_eyes,
                'c_lip_lst': c_lip,
                'exp': exp,
                'x_s': x_s,
                'kp': kp
            }],

        }
    }

In [11]:
def interpolate_descriptors_linear(descriptor_1, descriptor_2, num_intermediate_clusters=1):

    # Flatten the descriptors using the extract_vectors function
    flattened_1 = extract_full_vector(descriptor_1)
    flattened_2 = extract_full_vector(descriptor_2)

    # Interpolate between the two flattened descriptors
    intermediate_vectors = []
    for t in range(1, num_intermediate_clusters + 1):  # Generate specified number of intermediate vectors
        alpha = t / (num_intermediate_clusters + 1)  # Adjust alpha to control the number of intermediate clusters
        intermediate_vector = (1 - alpha) * flattened_1 + alpha * flattened_2
        intermediate_vectors.append(unflatten_vector(intermediate_vector))

    return descriptor_1, descriptor_2, intermediate_vectors

def interpolate_descriptors_spherical(descriptor_1, descriptor_2, num_intermediate_clusters=1):
    # Flatten the descriptors using the extract_vectors function
    flattened_1 = extract_full_vector(descriptor_1)
    flattened_2 = extract_full_vector(descriptor_2)
    
    # Compute the original norms of the descriptors
    norm_1 = np.linalg.norm(flattened_1)
    norm_2 = np.linalg.norm(flattened_2)
    
    if norm_1 == 0 or norm_2 == 0:
        raise ValueError("One of the input vectors has zero norm, cannot perform Slerp.")
    
    # Normalize the vectors for Slerp
    flattened_1_normalized = flattened_1 / norm_1
    flattened_2_normalized = flattened_2 / norm_2

    # Compute the angle between the two vectors
    dot_product = np.clip(np.dot(flattened_1_normalized, flattened_2_normalized), -1.0, 1.0)  # Ensure valid range
    theta = np.arccos(dot_product)  # Angle between vectors

    # Interpolate using spherical linear interpolation
    intermediate_vectors = []
    for t in range(1, num_intermediate_clusters + 1):  # Generate specified number of intermediate vectors
        alpha = t / (num_intermediate_clusters + 1)  # Interpolation parameter
        sin_theta = np.sin(theta)
        if sin_theta == 0:
            # Handle the case where the two vectors are nearly identical
            intermediate_vector = flattened_1_normalized
        else:
            intermediate_vector = (np.sin((1 - alpha) * theta) / sin_theta) * flattened_1_normalized + \
                                  (np.sin(alpha * theta) / sin_theta) * flattened_2_normalized
        
        # Scale the interpolated vector back to the original scale
        interpolated_norm = (1 - alpha) * norm_1 + alpha * norm_2  # Interpolated magnitude
        intermediate_vector = intermediate_vector * interpolated_norm  # Restore original scale
        
        intermediate_vectors.append(unflatten_vector(intermediate_vector))

    return descriptor_1, descriptor_2, intermediate_vectors

In [12]:
# file_path = f'descriptor_list.pkl'
# # Open the file in binary read mode
# with open(file_path, 'rb') as file:
#     descriptor_list = pickle.load(file)

experiment_path = "cluster_analysis/nov/18/23_18_56_10000_0.6_0.1_0.1_0.1_0.1"
with open(f"{experiment_path}/averaged_descriptors_raw.pkl", 'rb') as file:
    average_descriptor_dict = pickle.load(file)

In [16]:
import os

def read_cluster_ids_from_subfolders(base_folder):
    cluster_data = {}
    
    # Iterate through each subfolder in the base folder
    for subfolder_name in os.listdir(base_folder):
        subfolder_path = os.path.join(base_folder, subfolder_name)
        
        # Check if it's a directory
        if os.path.isdir(subfolder_path):
            file_path = os.path.join(subfolder_path, 'cluster_ids.txt')
            
            # Check if the 'cluster_ids.txt' file exists
            if os.path.isfile(file_path):
                with open(file_path, 'r') as file:
                    # Read and store the IDs as a list of integers
                    cluster_data[subfolder_name] = [average_descriptor_dict[int(line.strip())] for line in file if line.strip().isdigit()]
    
    return cluster_data

# Example usage
base_folder_path = 'video_generation_test/nov/18/23_18_56_10000_0.6_0.1_0.1_0.1_0.1/video_results/'
video_cluster_ids_dict = read_cluster_ids_from_subfolders(base_folder_path)

# # Print the result
# for subfolder, ids in cluster_ids_dict.items():
#     print(f"Subfolder: {subfolder}, Cluster IDs: {ids}")


In [23]:
for video in video_cluster_ids_dict:
    descriptor_list = video_cluster_ids_dict[video]
    new_list = []
    for i in range(0, len(descriptor_list)):
        new_list.append(descriptor_list[i])

    final_descriptor_list = []
    final_descriptor_list.append(new_list[0])

    for i in range(len(new_list) - 1):
        descriptor_1 = final_descriptor_list[-1]
        descriptor_2 = new_list[i + 1]
        descriptor_1, descriptor_2, intermediate_vectors = interpolate_descriptors_linear(descriptor_1, descriptor_2, num_intermediate_clusters=2)
        final_descriptor_list = final_descriptor_list + intermediate_vectors
        final_descriptor_list.append(descriptor_2)
    dir = "video_generation_test/video_gen_descriptor_8"
    os.makedirs(dir, exist_ok=True)
    print(f"{dir}/{video}.pkl")
    with open(f"{dir}/{video}.pkl", 'wb') as file:
        pickle.dump(final_descriptor_list, file)


video_generation_test/video_gen_descriptor_8/01-01-06-01-02-01-13.pkl
video_generation_test/video_gen_descriptor_8/01-01-02-02-01-01-07.pkl
video_generation_test/video_gen_descriptor_8/01-01-03-01-02-01-13.pkl
video_generation_test/video_gen_descriptor_8/01-01-03-01-01-02-18.pkl
video_generation_test/video_gen_descriptor_8/01-01-03-02-01-01-14.pkl
video_generation_test/video_gen_descriptor_8/01-01-02-02-02-02-02.pkl
video_generation_test/video_gen_descriptor_8/01-01-02-01-01-02-12.pkl
video_generation_test/video_gen_descriptor_8/02-01-03-01-02-01-13.pkl
video_generation_test/video_gen_descriptor_8/01-01-03-02-02-02-09.pkl
video_generation_test/video_gen_descriptor_8/01-01-03-01-02-02-11.pkl


# Spherical Interpolation

In [9]:
final_descriptor_list = []
final_descriptor_list.append(new_list[0])

for i in range(len(new_list) - 1):
    descriptor_1 = final_descriptor_list[-1]
    descriptor_2 = new_list[i + 1]
    descriptor_1, descriptor_2, intermediate_vectors = interpolate_descriptors_spherical(descriptor_1, descriptor_2, num_intermediate_clusters=3)
    final_descriptor_list = final_descriptor_list + intermediate_vectors
    final_descriptor_list.append(descriptor_2)

NameError: name 'new_list' is not defined

In [143]:
len(final_descriptor_list)
with open("video_descriptors_spherical.pkl", 'wb') as file:
    pickle.dump(descriptor_list, file)

# Interpolation using PCA vectors

In [ ]:
path = "cluster_analysis/nov/20/21_52_51_10000_0.8_0.05_0.05_0.05_0.05"

# Video save

In [25]:
import cv2
import os

def images_to_video(image_folder, output_video, fps=30):
    """
    Converts a sequence of images into a video.
    
    :param image_folder: Path to the folder containing image frames
    :param output_video: Path for the output video file
    :param fps: Frames per second for the video
    """
    # Get all image files in the folder
    images = [img for img in os.listdir(image_folder) if img.endswith((".png", ".jpg", ".jpeg"))]
    images.sort()  # Ensure files are sorted in sequence
    print(images)

    # Check if there are images
    if not images:
        print("No images found in the folder.")
        return
    
    # Get the size of the first image (assuming all images are the same size)
    first_image_path = os.path.join(image_folder, images[0])
    frame = cv2.imread(first_image_path)
    height, width, layers = frame.shape

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 'mp4v' for .mp4 files
    video = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    # Read and write each image to the video
    for image in images:
        image_path = os.path.join(image_folder, image)
        frame = cv2.imread(image_path)
        video.write(frame)

    # Release the video writer
    video.release()
    print(f"Video saved as {output_video}")

# Example usage:
def process_all_folders(input_base_folder, output_base_folder, fps=30):
    """
    Convert images to videos for all folders in the input base folder.
    
    Args:
        input_base_folder: Base folder containing subfolders with images
        output_base_folder: Base folder where output videos will be saved
        fps: Frames per second for the output videos
    """
    # Create output base folder if it doesn't exist
    os.makedirs(output_base_folder, exist_ok=True)
    
    # Get all subfolders in the input base folder
    subfolders = [f for f in os.listdir(input_base_folder) 
                  if os.path.isdir(os.path.join(input_base_folder, f))]
    
    for folder in subfolders:
        input_folder = os.path.join(input_base_folder, folder)
        output_video = os.path.join(output_base_folder, f"{folder}.mp4")
        
        print(f"Processing folder: {folder}")
        images_to_video(input_folder, output_video, fps)

# Example usage:
input_base = "video_generation_test/temp/fourth_batch"
output_base = "video_generation_test/temp/fourth_batch_videos" 
process_all_folders(input_base, output_base, fps=60)


Processing folder: 01-01-06-01-02-01-13
['processed_0.jpg', 'processed_1.jpg', 'processed_10.jpg', 'processed_100.jpg', 'processed_101.jpg', 'processed_102.jpg', 'processed_103.jpg', 'processed_104.jpg', 'processed_105.jpg', 'processed_106.jpg', 'processed_107.jpg', 'processed_108.jpg', 'processed_109.jpg', 'processed_11.jpg', 'processed_110.jpg', 'processed_111.jpg', 'processed_112.jpg', 'processed_113.jpg', 'processed_114.jpg', 'processed_115.jpg', 'processed_116.jpg', 'processed_117.jpg', 'processed_118.jpg', 'processed_119.jpg', 'processed_12.jpg', 'processed_120.jpg', 'processed_121.jpg', 'processed_122.jpg', 'processed_123.jpg', 'processed_124.jpg', 'processed_125.jpg', 'processed_126.jpg', 'processed_127.jpg', 'processed_128.jpg', 'processed_129.jpg', 'processed_13.jpg', 'processed_130.jpg', 'processed_131.jpg', 'processed_132.jpg', 'processed_133.jpg', 'processed_134.jpg', 'processed_135.jpg', 'processed_136.jpg', 'processed_137.jpg', 'processed_138.jpg', 'processed_139.jpg', '

In [4]:
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.fx.all import crop

def square_crop_video(input_path, output_path):
    # Load the video
    video = VideoFileClip(input_path)
    
    # Calculate the center crop dimensions
    width, height = video.size
    if width > height:
        crop_size = height
        x_center = width // 2
        x1 = x_center - crop_size // 2
        x2 = x_center + crop_size // 2
        y1, y2 = 0, height
    else:
        crop_size = width
        y_center = height // 2
        y1 = y_center - crop_size // 2
        y2 = y_center + crop_size // 2
        x1, x2 = 0, width
    
    # Crop the video to a square
    cropped_video = crop(video, x1=x1, y1=y1, x2=x2, y2=y2)
    
    # Write the output to a file
    cropped_video.write_videofile(output_path, codec="libx264", audio_codec="aac")

# Example usage
input_video_path = "video_generation_test/01-01-02-02-02-02-02.mp4"  # Replace with your input video path
output_video_path = "video_generation_test/output_video.mp4"  # Replace with your output video path
square_crop_video(input_video_path, output_video_path)


Moviepy - Building video video_generation_test/output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video video_generation_test/output_video.mp4



Moviepy - Done !
Moviepy - video ready video_generation_test/output_video.mp4
